In [13]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [14]:
# Constants for SVC model output
EMPTY = True
NOT_EMPTY = False

# Load the SVC model
MODEL = pickle.load(open("model.p", "rb"))

In [15]:
# Input data
mask_path = 'mask_1920_1080.png'
video_path = 'parking_1920_1080_loop.mp4'

# Read the mask
mask = cv2.imread(mask_path, 0)

# Capture video from the specified path
cap = cv2.VideoCapture(video_path)

# Obtain connected components to find parking spots
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

In [16]:
from skimage.transform import resize

# Function to predict if a parking spot is empty or not
def empty_or_not(spot_bgr):
    flat_data = []
    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY

# Function to get parking spot bounding boxes from connected components
def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values, centroids) = connected_components
    slots = []
    coef = 1  # Scaling coefficient

    for i in range(1, totalLabels):
        # Extract the bounding box coordinates
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)
        slots.append([x1, y1, w, h])

    return slots

In [17]:
# Video output path and settings
output_video_path = 'processed_parking_lot_video.avi'
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_rate = cap.get(cv2.CAP_PROP_FPS)

In [18]:
# Define the VideoWriter to save the output video
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, (frame_width, frame_height))

# Get parking spot bounding boxes
spots = get_parking_spots_bboxes(connected_components)

In [19]:
# Initialize variables for the processing loop
spots_status = [None for _ in spots]
diffs = [None for _ in spots]
previous_frame = None

frame_nmr = 0
ret = True
step = 30

In [20]:
# Process the video frame-by-frame
while ret:
    ret, frame = cap.read()

    # If a valid frame is read
    if ret:
        # Every 'step' frames, calculate the difference from the previous frame
        if frame_nmr % step == 0 and previous_frame is not None:
            for spot_idx, spot in enumerate(spots):
                x1, y1, w, h = spot
                spot_crop = frame[y1:y1 + h, x1:x1 + w]
                diffs[spot_idx] = np.abs(np.mean(spot_crop) - np.mean(previous_frame[y1:y1 + h, x1:x1 + w]))

        # Determine spot status based on differences
        if frame_nmr % step == 0:
            if previous_frame is None:
                arr_ = range(len(spots))
            else:
                arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]

            for spot_idx in arr_:
                spot = spots[spot_idx]
                x1, y1, w, h = spot
                spot_crop = frame[y1:y1 + h, x1:x1 + w]
                spot_status = empty_or_not(spot_crop)
                spots_status[spot_idx] = spot_status

        # Keep a copy of the frame for comparison
        if frame_nmr % step == 0:
            previous_frame = frame.copy()

        # Draw rectangles around parking spots based on their status
        for spot_idx, spot in enumerate(spots):
            x1, y1, w, h = spot
            if spots_status[spot_idx]:
                frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
            else:
                frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

        # Write text and draw on the frame
        cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
        cv2.putText(frame, f'Available spots: {sum(spots_status)} / {len(spots_status)}', (100, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Save the processed frame to the output video
        video_writer.write(frame)

    frame_nmr += 1

cap.release()
video_writer.release()